In [10]:
import pandas as pd
import ast
import os
from pymongo import MongoClient
from pymongo.errors import BulkWriteError
import ipywidgets as widgets
from IPython.display import display, clear_output
import re

# Al inicio de la celda 1, después de las importaciones
if 'widgets_initialized' not in globals():
    globals()['widgets_initialized'] = True
    # ... resto del código de widgets
else:
    print("⚠️ Widgets ya inicializados. Usa 'del widgets_initialized' para reinicializar.")

# Conexión MongoDB
client = MongoClient(os.environ["ATLASMONGODB_CONNECTION_STRING"])

# Colecciones
db = client["fraud_db"]
collection = db["transactions"]
productos_col = db["productos"]  # nueva colección para maestro

def clean_widget_callbacks(widget):
    """Limpia todos los callbacks de un widget"""
    if hasattr(widget, '_click_handlers'):
        widget._click_handlers.callbacks.clear()
    if hasattr(widget, '_observe_callbacks'):
        widget._observe_callbacks.clear()

# ---------------- Widgets de carga ----------------
file_picker = widgets.FileUpload(accept='.xlsx', multiple=False)
ruta_textbox = widgets.Text(value='', description='Archivo:', disabled=True)
btn_subir = widgets.Button(description="Subir a MongoDB", button_style='success')
progress = widgets.IntProgress(value=0, min=0, max=100, description='Progreso:')
output = widgets.Output()

# ---------------- Widgets de análisis ----------------
btn_generar_maestro = widgets.Button(description="Generar Maestro", button_style='info')
btn_calcular_top = widgets.Button(description="Top Productos Vendidos", button_style='warning')
maestro_output = widgets.Output()

# ---------------- Eventos de carga ----------------
def on_file_selected(change):
    with output:
        output.clear_output()
        if file_picker.value:
            nombre_archivo = file_picker.value[0]['name']
            ruta_textbox.value = nombre_archivo

# Limpiar callbacks anteriores antes de asignar nuevos
clean_widget_callbacks(file_picker)
file_picker.observe(on_file_selected, names='value')

def on_subir_clicked(b):
    with output:
        output.clear_output()

        if not file_picker.value:
            print("⚠️ Por favor selecciona un archivo .xlsx.")
            return

        uploaded_file = file_picker.value[0]
        contenido = uploaded_file['content']
        nombre_archivo = uploaded_file['name']
        with open("temp.xlsx", "wb") as f:
            f.write(contenido)

        df = pd.read_excel("temp.xlsx", engine="openpyxl")
        df['ITEM'] = df['ITEM'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
        df = df.rename(columns={'ID_Transaccion': '_id', 'ITEM': 'items'})
        registros = df.to_dict(orient="records")

        batch_size = 100
        total = len(registros)
        progress.max = total
        progress.value = 0
        insertados = 0

        for i in range(0, total, batch_size):
            batch = registros[i:i+batch_size]
            try:
                res = collection.insert_many(batch, ordered=False)
                insertados += len(res.inserted_ids)
            except BulkWriteError as bwe:
                errores = bwe.details.get("writeErrors", [])
                insertados += len(batch) - len(errores)
            progress.value = min(i + batch_size, total)

        print(f"✅ Procesados: {total} registros")
        print(f"📥 Insertados exitosamente: {insertados} registros (omitidos duplicados si hubo)")

# Limpiar callbacks anteriores antes de asignar nuevos
clean_widget_callbacks(btn_subir)
btn_subir.on_click(on_subir_clicked)

# ---------------- Maestro de productos ----------------
def on_generar_maestro(b):
    with maestro_output:
        maestro_output.clear_output()
        print("🔄 Generando maestro de productos...")
        
        transacciones = list(collection.find({}, {"_id": 0, "items": 1}))
        items_flat = [item.strip() for t in transacciones for item in t['items']]
        unique_items = sorted(set(items_flat))

        productos = []
        for item in unique_items:
            match = re.match(r"(.+?)_(\d+)$", item.strip())
            if match:
                nombre, codigo = match.groups()
                categoria = nombre.strip()
            else:
                nombre = item.strip()
                codigo = "NA"
                categoria = "otros"
            productos.append({
                "codigo": codigo,
                "nombre": item.strip(),
                "categoria": categoria
            })

        productos_col.delete_many({})
        productos_col.insert_many(productos)

        print(f"🎉 Maestro generado con {len(productos)} productos distintos.")
        df_prod = pd.DataFrame(productos)
        display(df_prod.groupby("categoria")["nombre"].count().reset_index(name='cantidad'))

# Limpiar callbacks anteriores antes de asignar nuevos
clean_widget_callbacks(btn_generar_maestro)
btn_generar_maestro.on_click(on_generar_maestro)

# ---------------- Top productos ----------------
def on_calcular_top(b):
    with maestro_output:
        maestro_output.clear_output()
        print("🔄 Calculando top productos...")
        
        transacciones = list(collection.find({}, {"_id": 0, "items": 1}))
        items_flat = [item.strip() for t in transacciones for item in t['items']]
        top = pd.Series(items_flat).value_counts().head(10)

        print("🏆 Top 10 productos más vendidos:")
        display(top.reset_index().rename(columns={'index': 'Producto', 0: 'Cantidad'}))

# Limpiar callbacks anteriores antes de asignar nuevos
clean_widget_callbacks(btn_calcular_top)
btn_calcular_top.on_click(on_calcular_top)

In [9]:
# ---------------- Mostrar interfaz ----------------
display(widgets.HTML("<h3>📤 Subir archivo Excel de transacciones</h3>"))
display(file_picker)
display(ruta_textbox)
display(btn_subir)
display(progress)
display(output)
display(widgets.HTML("<h3>🛒 Maestro y Análisis Exploratorio</h3>"))
display(widgets.HBox([btn_generar_maestro, btn_calcular_top]))
display(maestro_output)


HTML(value='<h3>📤 Subir archivo Excel de transacciones</h3>')

FileUpload(value=(), accept='.xlsx', description='Upload')

Text(value='', description='Archivo:', disabled=True)

Button(button_style='success', description='Subir a MongoDB', style=ButtonStyle())

IntProgress(value=0, description='Progreso:')

Output()

HTML(value='<h3>🛒 Maestro y Análisis Exploratorio</h3>')

Output()